##Bibliotecas

In [2]:
pip install py3dbp


  Preparing metadata (setup.py) ... done
  Created wheel for py3dbp: filename=py3dbp-1.1.2-py3-none-any.whl size=4881 sha256=cd9feb62060c02a59ba0571db1f9bc3ef789f832007bbf81a8a666d5d56ee680
  Stored in directory: /root/.cache/pip/wheels/f1/d7/0d/019e108d894325e502682cc704753a713ff2436a26b2409b31
Successfully built py3dbp


In [3]:
pip install plotly

## resposta com 3 digitos

In [ ]:
from py3dbp import Packer, Bin, Item
import re

# Função para coletar informações sobre as caixas
def coletar_informacoes_caixa(numero_caixas):
    informacoes_caixas = []

    for i in range(numero_caixas):
        print(f"\nCaixa {i+1}:")
        altura = float(input("Qual é a altura da caixa em centímetros? "))
        largura = float(input("Qual é a largura da caixa em centímetros? "))
        profundidade = float(input("Qual é a profundidade da caixa em centímetros? "))
        peso = float(input("Qual é o peso da caixa em quilogramas? "))

        print("\nEm uma escala de 1 a 5, o quão frágil é a caixa?")
        while True:
            fragilidade = float(input("Digite um número de 1 a 5 (sendo 1 menos frágil e 5 mais frágil): "))
            if 1 <= fragilidade <= 5:
                break
            else:
                print("Valor inválido! Digite um número entre 1 e 5.")

        informacoes_caixa = {
            "altura": altura,
            "largura": largura,
            "profundidade": profundidade,
            "peso": peso,
            "fragilidade": fragilidade
        }

        informacoes_caixas.append(informacoes_caixa)

    return informacoes_caixas

# Função para coletar informações sobre o recipiente (bin) e calcular o peso do bin
def coletar_informacoes_bin(caixas):
    bin = {}
    altura = float(input("Qual é a altura do bin em centímetros? "))
    largura = float(input("Qual é a largura do bin em centímetros? "))
    profundidade = float(input("Qual é a profundidade do bin em centímetros? "))

    # Calcular o peso do bin como 10 vezes o peso da caixa mais pesada
    peso_caixa_mais_pesada = max(caixas, key=lambda x: x["peso"])["peso"]
    peso_bin = peso_caixa_mais_pesada * 10

    bin["altura"] = altura
    bin["largura"] = largura
    bin["profundidade"] = profundidade
    bin["peso"] = peso_bin

    return bin

# Função para empacotar as caixas
def bin_packing(bin, caixas):
    packer = Packer()
    packer.add_bin(Bin('large-box', bin["largura"], bin["altura"], bin["profundidade"], bin["peso"]))

    for i, caixa in enumerate(caixas):
        packer.add_item(Item(f'caixa {i}', caixa["largura"], caixa["altura"], caixa["profundidade"], caixa["peso"]))

    packer.pack()

    responses = []

    for b in packer.bins:
        for item in b.items:
            responses.append(item.string())

    for item in b.unfitted_items:
        print("====> ", item.string())

    # Extrair os valores de x, y e z para cada caixa
    coordenadas_caixas = []
    for response in responses:
        # Usar expressão regular para extrair as coordenadas
        match = re.search(r'pos\((.*?)\)', response)
        if match:
            coordenadas_caixa = match.group(1)
            coordenadas_caixa = re.findall(r'-?\d+', coordenadas_caixa)[:3]  # Extrair os três primeiros números inteiros
            coordenadas_caixa = [int(coord) if coord else 0 for coord in coordenadas_caixa]  # Substituir por 0 se não houver número
            coordenadas_caixa += [0] * (3 - len(coordenadas_caixa))  # Adicionar zeros para garantir três números
            coordenadas_caixas.append(coordenadas_caixa)

    return coordenadas_caixas

# Função principal para empacotar caixas
def empacotar_caixas():
    # Coletar informações sobre as caixas
    num_caixas = int(input("Quantas caixas você deseja empacotar? "))
    caixas_info = coletar_informacoes_caixa(num_caixas)

    # Coletar informações sobre o recipiente (bin) e calcular o peso do bin
    bin_info = coletar_informacoes_bin(caixas_info)

    # Executar o empacotamento
    coordenadas_caixas = bin_packing(bin_info, caixas_info)

    # Imprimir as coordenadas das caixas empacotadas
    for i, coordenadas in enumerate(coordenadas_caixas):
        print(f'Caixa {i} - Posição (x, y, z): {coordenadas}')

# Chamar a função principal para empacotar as caixas
empacotar_caixas()



##grafico base

In [ ]:
import plotly.graph_objects as go

# Dimensões da caixa
x_length = 11000
y_length = 8500
z_length = 5500

# Pontos da caixa
x = [-x_length/2, -x_length/2, x_length/2, x_length/2, -x_length/2, -x_length/2, x_length/2, x_length/2]
y = [-y_length/2, y_length/2, y_length/2, -y_length/2, -y_length/2, y_length/2, y_length/2, -y_length/2]
z = [-z_length/2, -z_length/2, -z_length/2, -z_length/2, z_length/2, z_length/2, z_length/2, z_length/2]

# Definição das arestas da caixa
edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],  # Arestas da face inferior
    [4, 5], [5, 6], [6, 7], [7, 4],  # Arestas da face superior
    [0, 4], [1, 5], [2, 6], [3, 7],   # Arestas verticais
    [4, 6], [5, 7]                    # Arestas horizontais
]

# Criação do gráfico 3D interativo com a caixa
fig = go.Figure(data=[
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(color='blue', width=3)
    )
])

# Adição das arestas extras para conectar a caixa
for edge in edges:
    x_edge = [x[edge[0]], x[edge[1]]]
    y_edge = [y[edge[0]], y[edge[1]]]
    z_edge = [z[edge[0]], z[edge[1]]]
    fig.add_trace(go.Scatter3d(
        x=x_edge,
        y=y_edge,
        z=z_edge,
        mode='lines',
        line=dict(color='blue', width=3)
    ))

# Configuração dos rótulos dos eixos
fig.update_layout(
    scene=dict(
        xaxis=dict(title='Eixo X', range=[-15000, 15000]),
        yaxis=dict(title='Eixo Y', range=[-10000, 10000]),
        zaxis=dict(title='Eixo Z', range=[-7500, 7500]),
        aspectmode='cube'
    )
)

# Exibição do gráfico interativo
fig.show()


##coleta informações caixas

In [13]:
from py3dbp import Packer, Bin, Item
import re
def coletar_informacoes_caixa(numero_caixas):
    informacoes_caixas = []

    for i in range(numero_caixas):
        print(f"\nCaixa {i+1}:")
        altura = input("Qual é a altura da caixa em centímetros? ").replace(',', '.')
        largura = input("Qual é a largura da caixa em centímetros? ").replace(',', '.')
        profundidade = input("Qual é a profundidade da caixa em centímetros? ").replace(',', '.')
        peso = input("Qual é o peso da caixa em quilogramas? ").replace(',', '.')

        print("\nEm uma escala de 1 a 5, o quão frágil é a caixa?")
        while True:
            fragilidade = input("Digite um número de 1 a 5 (sendo 1 menos frágil e 5 mais frágil): ").replace(',', '.')
            try:
                fragilidade = float(fragilidade)
                if 1 <= fragilidade <= 5:
                    break
                else:
                    print("Valor inválido! Digite um número entre 1 e 5.")
            except ValueError:
                print("Valor inválido! Digite um número válido.")

        informacoes_caixa = {
            "altura": float(altura),
            "largura": float(largura),
            "profundidade": float(profundidade),
            "peso": float(peso),
            "fragilidade": fragilidade
        }

        informacoes_caixas.append(informacoes_caixa)

    return informacoes_caixas


In [5]:
def coletar():
    # Coletar informações sobre as caixas
    num_caixas = int(input("Quantas caixas você deseja empacotar? "))
    caixas_info = coletar_informacoes_caixa(num_caixas)

    #TO-DO add validation on parameters

    # Coletar informações sobre o recipiente (bin) e calcular o peso do bin
    bin_info = coletar_informacoes_bin(caixas_info)
    return (caixas_info, bin_info)


##coleta informações bin

In [21]:
def coletar_informacoes_bin(caixas):
    bin = {}
    altura = input("Qual é a altura do bin em centímetros? ").replace(',', '.')
    largura = input("Qual é a largura do bin em centímetros? ").replace(',', '.')
    profundidade = input("Qual é a profundidade do bin em centímetros? ").replace(',', '.')

    try:
        altura = float(altura)
        largura = float(largura)
        profundidade = float(profundidade)
    except ValueError:
        print("Valores inválidos! Certifique-se de digitar números válidos.")
        return None

    # Calcular o peso do bin como 10 vezes o peso da caixa mais pesada
    peso_caixa_mais_pesada = max(caixas, key=lambda x: x["peso"])["peso"]
    peso_bin = peso_caixa_mais_pesada * 10

    bin["altura"] = altura
    bin["largura"] = largura
    bin["profundidade"] = profundidade
    bin["peso"] = peso_bin

    return bin


##pensamento logico para coordenadas

In [22]:
def bin_packing(bin, caixas):
    packer = Packer()
    packer.add_bin(Bin('large-box', bin["largura"], bin["altura"], bin["profundidade"], bin["peso"]))

    for i, caixa in enumerate(caixas):
        packer.add_item(Item(f'caixa {i}', caixa["largura"], caixa["altura"], caixa["profundidade"], caixa["peso"]))

    packer.pack()

    responses = []

    for b in packer.bins:
        for item in b.items:
            responses.append(item.string())

    for item in b.unfitted_items:
        print("====> ", item.string())

    # Extrair os valores de x, y e z para cada caixa
    coordenadas_caixas = []
    for response in responses:
        # Usar expressão regular para extrair as coordenadas
        match = re.search(r'pos\((.*?)\)', response)
        if match:
            coordenadas_caixa = match.group(1)
            coordenadas_caixa = re.findall(r'-?\d+', coordenadas_caixa)[:3]  # Extrair os três primeiros números inteiros
            coordenadas_caixa = [int(coord) if coord else 0 for coord in coordenadas_caixa]  # Substituir por 0 se não houver número
            coordenadas_caixa += [0] * (3 - len(coordenadas_caixa))  # Adicionar zeros para garantir três números
            coordenadas_caixas.append(coordenadas_caixa)

    return coordenadas_caixas

##função perguntar para coletar os dados

In [23]:
caixas_info, bin_info = coletar()

Quantas caixas você deseja empacotar? 1

Caixa 1:
Qual é a altura da caixa em centímetros? 1
Qual é a largura da caixa em centímetros? 1
Qual é a profundidade da caixa em centímetros? 1
Qual é o peso da caixa em quilogramas? 1

Em uma escala de 1 a 5, o quão frágil é a caixa?
Digite um número de 1 a 5 (sendo 1 menos frágil e 5 mais frágil): 1
Qual é a altura do bin em centímetros? 15,7
Qual é a largura do bin em centímetros? 18,9
Qual é a profundidade do bin em centímetros? 17,6


##onde estao os dados

In [24]:
caixas_info

[{'altura': 1.0,
  'largura': 1.0,
  'profundidade': 1.0,
  'peso': 1.0,
  'fragilidade': 1.0}]

In [25]:
bin_info

{'altura': 15.7, 'largura': 18.9, 'profundidade': 17.6, 'peso': 10.0}

In [26]:
coordenadas_caixas = bin_packing(bin_info, caixas_info)

In [27]:
coordenadas_caixas

[[0, 0, 0]]

##não lembro o que faz mas n quero perder

In [ ]:
def plot_chart(coordenadas_caixas, caixas_info, bin_info):
    fig = go.Figure()

    # Adicionar as caixas empacotadas no gráfico
    for caixa_coordenadas, caixa in zip(coordenadas_caixas, caixas_info):
        x, y, z = caixa_coordenadas
        largura = caixa["largura"]
        profundidade = caixa["profundidade"]
        altura = caixa["altura"]
        fig.add_trace(go.Mesh3d(
            x=[x, x, x + largura, x + largura, x, x, x + largura, x + largura],


            z=[y, y + profundidade, y + profundidade, y, y, y + profundidade, y + profundidade, y],
            y=[z, z, z, z, z + altura, z + altura, z + altura, z + altura],
            opacity=0.5,
            color='blue'
        ))

    # Configuração dos rótulos dos eixos e limites com base nas dimensões do bin
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='Eixo X', range=[0, bin_info["largura"]]),
            yaxis=dict(title='Eixo Y', range=[0, bin_info["profundidade"]]),
            zaxis=dict(title='Eixo Z', range=[0,  bin_info["altura"]
]),
            aspectmode='cube'
        )
    )

    # Exibição do gráfico interativo
    fig.show()

##base do grafico

In [ ]:
def combinar_coordenadas(coordenadas_caixas, caixas_info, bin_info):



    # Elemento 1
    ([0, 4, 4, 0, 0, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0],
     [0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 3, 3, 3],
     [0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3],
     [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]),

    # Elemento 2
    ([10, 14, 14, 10, 10, 14, 14, 10, 10, 10, 14, 14, 14, 14, 10, 10],
     [10, 10, 13, 13, 10, 10, 13, 13, 10, 10, 10, 10, 13, 13, 13, 13],
     [10, 10, 10, 10, 13, 13, 13, 13, 10, 13, 10, 13, 10, 13, 10, 13],
     [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]),





##o que vai aparecer

In [ ]:
data_list = (
    [0, 4, 4, 0, 0, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0],
    [0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 3, 3, 3],
    [0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3],
    [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]
)


In [ ]:
from py3dbp import Packer, Bin, Item
import re
import plotly.graph_objects as go


def empacotar_caixas():
  caixas_info, bin_info = coletar()
  coordenadas_caixas = bin_packing(bin_info, caixas_info)
  data_list = combinar_coordenadas(coordenadas_caixas, caixas_info, bin_info)
  plot_multiple_3d_graphs(data_list)
empacotar_caixas()

Quantas caixas você deseja empacotar? 


ValueError: ignored

In [39]:
import plotly.graph_objects as go
import random

def plot_multiple_3d_graphs(data_list):
    # Criar a figura
    fig = go.Figure()

    for data in data_list:
        x, y, z, arestas = data
        color = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Adicionar os vértices
        fig.add_trace(go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers+text',
            marker=dict(size=4, color=color),
        ))

        # Adicionar as arestas
        for aresta in arestas:
            fig.add_trace(go.Scatter3d(
                x=[x[i] for i in aresta],
                y=[y[i] for i in aresta],
                z=[z[i] for i in aresta],
                mode='lines',
                line=dict(color=color, width=3)
            ))

    # Definir os intervalos dos eixos (ajuste conforme necessário)
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-1, 5]),
            yaxis=dict(range=[-1, 4]),
            zaxis=dict(range=[-1, 4]),
        )
    )

    # Definir os intervalos dos eixos
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, 30]),
            yaxis=dict(range=[0, 40]),
            zaxis=dict(range=[0, 50])
        )
    )

    # Mostrar a figura
    fig.show()

# Exemplo com 3 elementos na lista de dados
data_list = [
    # Elemento 1
    ([0, 4, 4, 0, 0, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0],
     [0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 3, 3, 3],
     [0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3],
     [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]),

    # Elemento 2
    ([10, 14, 14, 10, 10, 14, 14, 10, 10, 10, 14, 14, 14, 14, 10, 10],
     [10, 10, 13, 13, 10, 10, 13, 13, 10, 10, 10, 10, 13, 13, 13, 13],
     [10, 10, 10, 10, 13, 13, 13, 13, 10, 13, 10, 13, 10, 13, 10, 13],
     [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]),


]

plot_multiple_3d_graphs(data_list)


In [43]:
import plotly.graph_objects as go
import random

def coletar_informacoes_bin(caixas):
    bin = {}
    altura = input("Qual é a altura do bin em centímetros? ").replace(',', '.')
    largura = input("Qual é a largura do bin em centímetros? ").replace(',', '.')
    profundidade = input("Qual é a profundidade do bin em centímetros? ").replace(',', '.')

    try:
        altura = float(altura)
        largura = float(largura)
        profundidade = float(profundidade)
    except ValueError:
        print("Valores inválidos! Certifique-se de digitar números válidos.")
        return None

    # Calcular o peso do bin como 10 vezes o peso da caixa mais pesada
    peso_caixa_mais_pesada = max(caixas, key=lambda x: x[3])  # Acessar o peso diretamente na tupla
    peso_bin = peso_caixa_mais_pesada[3] * 10

    bin["altura"] = altura
    bin["largura"] = largura
    bin["profundidade"] = profundidade
    bin["peso"] = peso_bin

    return bin

def plot_multiple_3d_graphs(data_list, bin_info):
    # Criar a figura
    fig = go.Figure()

    for data in data_list:
        x, y, z, arestas = data
        color = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Adicionar os vértices
        fig.add_trace(go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers+text',
            marker=dict(size=4, color=color),
        ))

        # Adicionar as arestas
        for aresta in arestas:
            fig.add_trace(go.Scatter3d(
                x=[x[i] for i in aresta],
                y=[y[i] for i in aresta],
                z=[z[i] for i in aresta],
                mode='lines',
                line=dict(color=color, width=3)
            ))


    # Definir os intervalos dos eixos
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, bin_info["profundidade"]]),
            yaxis=dict(range=[0, bin_info["largura"]]),
            zaxis=dict(range=[0, bin_info["altura"]])
        )
    )

    # Mostrar a figura
    fig.show()

# Exemplo com 3 elementos na lista de dados
data_list = [
    # Elemento 1
    ([0, 4, 4, 0, 0, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0],
     [0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 3, 3, 3, 3],
     [0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3],
     [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]),

    # Elemento 2
    ([10, 14, 14, 10, 10, 14, 14, 10, 10, 10, 14, 14, 14, 14, 10, 10],
     [10, 10, 13, 13, 10, 10, 13, 13, 10, 10, 10, 10, 13, 13, 13, 13],
     [10, 10, 10, 10, 13, 13, 13, 13, 10, 13, 10, 13, 10, 13, 10, 13],
     [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (8, 9), (10, 11), (12, 13), (14, 15)]),
]

# Coletar informações do bin
bin_info = coletar_informacoes_bin(data_list)

# Plotar as caixas e o bin
plot_multiple_3d_graphs(data_list, bin_info)


Qual é a altura do bin em centímetros? 50
Qual é a largura do bin em centímetros? 40
Qual é a profundidade do bin em centímetros? 30


In [50]:
from py3dbp import Packer, Bin, Item
import re
import plotly.graph_objects as go
import random

def coletar_informacoes_caixa(numero_caixas):
    informacoes_caixas = []

    for i in range(numero_caixas):
        print(f"\nCaixa {i+1}:")
        altura = input("Qual é a altura da caixa em centímetros? ").replace(',', '.')
        largura = input("Qual é a largura da caixa em centímetros? ").replace(',', '.')
        profundidade = input("Qual é a profundidade da caixa em centímetros? ").replace(',', '.')
        peso = input("Qual é o peso da caixa em quilogramas? ").replace(',', '.')

        print("\nEm uma escala de 1 a 5, o quão frágil é a caixa?")
        while True:
            fragilidade = input("Digite um número de 1 a 5 (sendo 1 menos frágil e 5 mais frágil): ").replace(',', '.')
            try:
                fragilidade = float(fragilidade)
                if 1 <= fragilidade <= 5:
                    break
                else:
                    print("Valor inválido! Digite um número entre 1 e 5.")
            except ValueError:
                print("Valor inválido! Digite um número válido.")

        informacoes_caixa = {
            "altura": float(altura),
            "largura": float(largura),
            "profundidade": float(profundidade),
            "peso": float(peso),
            "fragilidade": fragilidade
        }

        informacoes_caixas.append(informacoes_caixa)

    return informacoes_caixas

# Resto do código permanece igual

# Corrigir a função plot_multiple_3d_graphs
def plot_multiple_3d_graphs(data_list, bin_info):
    # Criar a figura
    fig = go.Figure()

    for data in data_list:
        x, y, z = data  # Agora só temos 3 valores: largura, profundidade, altura
        color = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Adicionar os vértices
        fig.add_trace(go.Scatter3d(
            x=[0, x, x, 0, 0, x, x, 0, 0, 0, x, x, x, x, 0, 0],
            y=[0, 0, y, y, 0, 0, y, y, 0, 0, 0, 0, y, y, y, y],
            z=[0, 0, 0, 0, z, z, z, z, 0, z, 0, z, 0, z, 0, z],
            mode='markers+text',
            marker=dict(size=4, color=color),
        ))

    # Definir os intervalos dos eixos
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, bin_info["profundidade"]]),
            yaxis=dict(range=[0, bin_info["largura"]]),
            zaxis=dict(range=[0, bin_info["altura"]])
        )
    )

    # Mostrar a figura
    fig.show()

# Coletar informações das caixas
numero_caixas = int(input("Quantas caixas deseja adicionar? "))
informacoes_caixas = coletar_informacoes_caixa(numero_caixas)

# Coletar informações do bin
bin_info = coletar_informacoes_bin(informacoes_caixas)

# Plotar as caixas e o bin
plot_multiple_3d_graphs([(caixa["largura"], caixa["profundidade"], caixa["altura"]) for caixa in informacoes_caixas], bin_info)


Quantas caixas deseja adicionar? 2

Caixa 1:
Qual é a altura da caixa em centímetros? 15
Qual é a largura da caixa em centímetros? 14
Qual é a profundidade da caixa em centímetros? 13
Qual é o peso da caixa em quilogramas? 20

Em uma escala de 1 a 5, o quão frágil é a caixa?
Digite um número de 1 a 5 (sendo 1 menos frágil e 5 mais frágil): 1

Caixa 2:
Qual é a altura da caixa em centímetros? 25
Qual é a largura da caixa em centímetros? 26
Qual é a profundidade da caixa em centímetros? 24
Qual é o peso da caixa em quilogramas? 30

Em uma escala de 1 a 5, o quão frágil é a caixa?
Digite um número de 1 a 5 (sendo 1 menos frágil e 5 mais frágil): 1
Qual é a altura do bin em centímetros? 50
Qual é a largura do bin em centímetros? 40
Qual é a profundidade do bin em centímetros? 30


##outra coisa


In [35]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Função para treinar um modelo local em um dispositivo federado
def treinar_modelo_local(X, y):
    modelo_local = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(3,)),  # Três atributos para as dimensões das caixas
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # Três classes: pequeno, médio e grande
    ])

    modelo_local.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    modelo_local.fit(X, y, epochs=50, batch_size=2, verbose=0)

    return modelo_local

# Dados de treinamento fictícios para dois dispositivos federados (exemplo)
dados_loja1 = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])
categorias_loja1 = np.array([0, 1, 1])  # Categorias codificadas como números inteiros

dados_loja2 = np.array([[4, 5, 6], [2, 3, 4], [3, 4, 5]])
categorias_loja2 = np.array([2, 1, 1])  # Categorias codificadas como números inteiros

# Treinamento de modelos locais para cada dispositivo federado
modelo_loja1 = treinar_modelo_local(dados_loja1, categorias_loja1)
modelo_loja2 = treinar_modelo_local(dados_loja2, categorias_loja2)

# Agregação de modelos locais para criar um modelo global
def criar_modelo_global(modelos_locais):
    # Combine os pesos de todos os modelos locais em uma lista
    todos_pesos = [modelo.get_weights() for modelo in modelos_locais]

    # Calcule a média dos pesos
    pesos_globais = [np.mean(peso_local, axis=0) for peso_local in zip(*todos_pesos)]

    # Crie um modelo global com a mesma arquitetura do primeiro modelo local
    modelo_global = treinar_modelo_local(dados_loja1, categorias_loja1)

    # Atribua os pesos globais ao modelo global
    modelo_global.set_weights(pesos_globais)

    return modelo_global

# Agregação de modelos locais para criar um modelo global
modelos_locais = [modelo_loja1, modelo_loja2]
modelo_global = criar_modelo_global(modelos_locais)

# Exemplo de uso do modelo global
caixas_a_organizar = np.array([[2, 3, 4], [3, 4, 5], [1, 2, 3]])
categorias_previstas = modelo_global.predict(caixas_a_organizar)
categorias_previstas_decodificadas = encoder.inverse_transform(np.argmax(categorias_previstas, axis=1))

print("Previsões para as caixas usando o modelo global:")
for caixa, previsao in zip(caixas_a_organizar, categorias_previstas_decodificadas):
    print(f"Caixa {caixa} -> Categoria: {previsao}")


1/1 [==============================] - 0s 72ms/step
Previsões para as caixas usando o modelo global:
Caixa [2 3 4] -> Categoria: médio
Caixa [3 4 5] -> Categoria: médio
Caixa [1 2 3] -> Categoria: médio
